In [ ]:
import numpy as np

from scipy.integrate import solve_ivp

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "seaborn"

# Brusselator model

The dynamics of the oscillating reaction discovered by Belousov and Zhabotinsky,
can be modeled through the so-called Brusselator model depending on two parameters:

$$
\left\{\begin{aligned}
{\mathrm d}_t y_1 & = a - (b+1) y_1 + y_1^2y_2\\
{\mathrm d}_t y_2 & = b y_1 - y_1^2y_2
\end{aligned}\right.
$$

For this PC on equlibrium study and continuation, we will set $b=3$ and work with $a$ as a bifurcation parameter in a neighbor of $\sqrt{2}$, for which a Hopf bifurcation is taking place.

In [ ]:
b = 3

def f(y):
    a, y1, y2 = y
    y1_dot = a - (b+1)*y1 + y1*y1*y2 
    y2_dot = b*y1 - y1*y1*y2  
    return np.array([y1_dot, y2_dot])
             
# specific form of f to use solve_ivp method
def f2(t, y, a):
    y1, y2 = y
    return f((a, y1,y2))

def df(y):
    a, y1, y2 = y
    return np.array([[-(b+1) + 2*y1*y2, y1*y1], [b - 2*y1*y2, -y1*y1]])

def dfda(y):
    a, y1, y2 = y
    return np.array([1, 0])

def jac_eq(a):
    return np.array([[b-1, a*a], [-b, -a*a]])

In [ ]:
a = np.sqrt(2)-0.7

tini = 0. 
tend = 80.

yini = (1.5, 0)

tol = 1.e-10
sol = solve_ivp(f2, (tini, tend), yini, method="RK45", args=(a,), rtol=tol, atol=tol)
eig_vals_eq = np.linalg.eigvals(jac_eq(a))

fig = make_subplots(rows=2, cols=2, specs=[[{"colspan": 2}, None], [{}, {}]],
                    subplot_titles=("Solutions", "Phase portrait", "Eigenvalues at equilibrium points"))

fig.add_trace(go.Scatter(x=sol.t, y=sol.y[0], name='y1'), row=1, col=1)
fig.add_trace(go.Scatter(x=sol.t, y=sol.y[1], name='y2'), row=1, col=1)

fig.add_trace(go.Scatter(x=sol.y[0], y=sol.y[1], showlegend=False), row=2, col=1)

fig.add_trace(go.Scatter(x=np.real(eig_vals_eq) , y=np.imag(eig_vals_eq), showlegend=False, 
                         mode='markers', marker=dict(symbol='x', size=10)), row=2, col=2)
fig.add_vline(x=0.0, line_width=1, row=2, col=2)
fig.add_hline(y=0.0, line_width=1, row=2, col=2)

steps = []
for ai in np.arange(np.sqrt(2)-0.7, np.sqrt(2)+0.6, 0.1):
    eig_vals_eq = np.linalg.eigvals(jac_eq(ai))
    sol = solve_ivp(f2, (tini, tend), yini, method="RK45", args=(ai,), rtol=tol, atol=tol)
    args = [{"x": [sol.t, sol.t, sol.y[0], np.real(eig_vals_eq)], 
             "y": [sol.y[0], sol.y[1], sol.y[1], np.imag(eig_vals_eq)]}]
    step = dict(method="update", label = f"{ai:.2f}", args=args)
    steps.append(step)
sliders = [dict(currentvalue={'prefix': 'a = '}, steps=steps)]

fig.update_layout(sliders=sliders, height=750)
fig.update_xaxes(title_text="t", row=1)
fig.update_xaxes(title_text="y1", col=1, row=2)
fig.update_yaxes(title_text="y2", col=1, row=2)
fig.update_xaxes(range=[-1,1], col=2, row=2)
fig.update_yaxes(range=[-2,2], col=2, row=2)
fig['layout']['sliders'][0]['pad']=dict(t=50)

fig.show()

## Natural continuation

In [ ]:
def nat_cont_trivial_pred(sol_ini, npts, dlamb, f, df, tol=1.e-8, nmax_newton=100):
        
    print("Natural continuation (trivial prediction)")
            
    sol = np.zeros((len(sol_ini), npts))
    sol[:,0] = sol_ini
    
    is_stable = np.zeros(npts, dtype=bool)
    eig_vals_eq = np.linalg.eigvals(df(sol_ini))
    is_stable[0] = np.all(np.real(eig_vals_eq) < 0)
    
    nb_iter_newton = 0
        
    for it in range(npts-1):
               
        # Newton iteration
        soln = np.copy(sol[:, it])
        soln[0] = soln[0] + dlamb
        #print("Iter nb ", 0, " ||f(y)|| = ",  np.linalg.norm(f(soln)))
        for it_newton in range(nmax_newton):
            soln[1:] = soln[1:] + np.linalg.solve(df(soln), -f(soln))
            #print("Iter nb ", it_newton+1, " ||f(y)|| = ",  np.linalg.norm(f(soln)))
            
            if (np.linalg.norm(f(soln)) < tol):
                nb_iter_newton += it_newton+1
                #print(it_newton+1)
                break
                
        if (np.linalg.norm(f(soln)) > tol):
            warnings.warn(f"Newton may not have converged")
            return sol[:,:it+1], is_stable[:it+1]
        
        eig_vals = np.linalg.eigvals(df(soln))
        is_stable[it+1] = np.all(np.real(eig_vals)<0)
        sol[:, it+1] = soln
        
    print(f"Average number of newton iterations per step : {nb_iter_newton/(npts-1)}")
        
    return sol, is_stable

In [ ]:
def nat_cont_linear_pred(sol_ini, npts, dlamb, f, df, dfdlambda, tol=1.e-8, nmax_newton=100):
    
    print("Natural continuation (linear prediction)")
            
    sol = np.zeros((len(sol_ini), npts))
    sol[:,0] = sol_ini
    
    is_stable = np.zeros(npts, dtype=bool)
    eig_vals_eq = np.linalg.eigvals(df(sol_ini))
    is_stable[0] = np.all(np.real(eig_vals_eq) < 0)
    
    nb_iter_newton = 0
        
    for it in range(npts-1):
               
        # Newton iteration
        soln = np.copy(sol[:, it])
        der = -np.linalg.solve(df(soln),dfdlambda(soln))
        soln[0] = soln[0] + dlamb
        soln[1:] = soln[1:] + der * dlamb
        #print("Iter nb ", 0, " ||f(y)|| = ",  np.linalg.norm(f(soln)))
        for it_newton in range(nmax_newton):
            soln[1:] = soln[1:] + np.linalg.solve(df(soln), -f(soln))
            #print("Iter nb ", it_newton+1, " ||f(y)|| = ",  np.linalg.norm(f(soln)))
            
            if (np.linalg.norm(f(soln)) < tol):
                nb_iter_newton += it_newton+1
                #print(it_newton+1)
                break
                
        if (np.linalg.norm(f(soln)) > tol):
            warnings.warn(f"Newton may not have converged")
            return sol[:,:it+1], is_stable[:it+1]
            
        eig_vals = np.linalg.eigvals(df(soln))
        is_stable[it+1] = np.all(np.real(eig_vals)<0)
        sol[:, it+1] = soln
        
    print(f"Average number of newton iterations per step : {nb_iter_newton/(npts-1)}")
        
    return sol, is_stable

In [ ]:
a_ini = 0.1

sol_ini = (a_ini, a_ini, 3/a_ini)

npts = 200

dlamb = 1e-2

sol, is_stable = nat_cont_trivial_pred(sol_ini, npts, dlamb, f, df)
# sol, is_stable = nat_cont_linear_pred(sol_ini, npts, dlamb, f, df, dfda)

fig = go.fig = make_subplots(rows=2, cols=1)
marker_unstab=dict(symbol='x', color='rgb(196,78,82)')
marker_stab=dict(symbol='x', color='rgb(85,168,104)')

fig.add_trace(go.Scatter(x=sol[0, is_stable], y=sol[1, is_stable], name='stable eq.', mode='markers', 
                         marker=marker_stab), row=1, col=1)
fig.add_trace(go.Scatter(x=sol[0, np.logical_not(is_stable)], y=sol[1, np.logical_not(is_stable)], 
                         name='unstable eq.', mode='markers', marker=marker_unstab), row=1, col=1)

fig.add_trace(go.Scatter(x=sol[0, is_stable], y=sol[2, is_stable], name='stable eq.', mode='markers', 
                         marker=marker_stab, showlegend=False), row=2, col=1)
fig.add_trace(go.Scatter(x=sol[0, np.logical_not(is_stable)], y=sol[2, np.logical_not(is_stable)], showlegend=False,
                         name='unstable eq.', mode='markers', marker=marker_unstab), row=2, col=1)

fig.update_xaxes(title_text="a")
fig.update_yaxes(title_text="y1", row=1)
fig.update_yaxes(title_text="y2", row=2)

fig.update_layout(height=750)
fig.show()